## Baseline mBART & fine-tuned mBART models



In [1]:
#install libraries
!pip install transformers # transformers==4.31.0
!pip install sacrebleu
!pip install sacremoses
!pip install sentencepiece

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00


In [2]:
!pip install datasets==2.13.1
!pip install evaluate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00


#Fine-tuning mBART model

In [4]:
#fine-tuning mBART model using 1k emea - training for 3 epochs

#facebook/mbart-large-50-one-to-many-mmt
# TODO redirect stdout to file
!python run_translation_no_trainer.py \
    --model_name_or_path facebook/mbart-large-50-one-to-many-mmt \
    --source_lang en_XX \
    --target_lang de_DE \
    --forced_bos_token de_DE \
    --learning_rate 3e-05 \
    --num_warmup_steps 500 \
    --num_train_epochs 3 \
    --num_beams 5 \
    --max_source_length 250 \
    --max_target_length 250 \
    --per_device_eval_batch_size 1 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --train_file en-de.emea.1k.train.json \
    --validation_file en-de.emea.500.valid.json \
    --fp16 \
    --output_dir models/mbart50_emea_3epoch &> log.txt


# Translating the Medline corpus

In [12]:
#translating medline using finetuned mBART

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
src_file = 'medline_en-de.en.txt'
output_file = "translation_finetuned_mBART.txt"
checkpoint = 'models/mbart50_emea_3epoch'
mnmt = 'de_DE'
src_lines = []
device = 'cuda'

#from: https://huggingface.co/docs/transformers/main/model_doc/mbart
model = MBartForConditionalGeneration.from_pretrained(checkpoint)
model = model.to(device)
tokenizer = MBart50TokenizerFast.from_pretrained(checkpoint)


tokenizer.src_lang = "en_XX"

with open(src_file, 'r', encoding='utf-8') as src, open(output_file, 'w', encoding='utf-8') as output:
    for line in src:
        line = line.strip()
        encoded = tokenizer(line, return_tensors="pt").to(device)
        generated_tokens = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.lang_code_to_id[mnmt]
        )
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        translated_text = translation[0] + '\n'
        output.write(translated_text)

In [ ]:
#translating medline using baseline mBART model

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
src_file = 'medline_en-de.en.txt'
output_file = "translation_baseline_mBART.txt"
checkpoint = 'facebook/mbart-large-50-one-to-many-mmt'
mnmt = 'de_DE'
src_lines = []
device = 'cuda'

#from: https://huggingface.co/docs/transformers/main/model_doc/mbart
model = MBartForConditionalGeneration.from_pretrained(checkpoint)
model = model.to(device)
tokenizer = MBart50TokenizerFast.from_pretrained(checkpoint)


tokenizer.src_lang = "en_XX"

with open(src_file, 'r', encoding='utf-8') as src, open(output_file, 'w', encoding='utf-8') as output:
    for line in src:
        line = line.strip()
        encoded = tokenizer(line, return_tensors="pt").to(device)
        generated_tokens = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.lang_code_to_id[mnmt]
        )
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        translated_text = translation[0] + '\n'
        output.write(translated_text)

#Evaluating models


In [3]:
!pip install sacrebleu

In [17]:
!pip install unbabel-comet
import comet

  Using cached unbabel_comet-2.2.0-py3-none-any.whl (92 kB)


In [13]:
#compute BLEU and chrF scores for fine-tuned model
!sacrebleu medline_en-de.de.txt -l en-de -i translation_finetuned_mBART.txt  -m bleu chrf

[
{
 "name": "BLEU",
 "score": 24.8,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.0",
 "verbose_score": "47.9/27.1/19.6/15.0 (BP = 1.000 ratio = 1.033 hyp_len = 1268 ref_len = 1228)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.0"
},
{
 "name": "chrF2",
 "score": 47.9,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:0|space:no|version:2.4.0",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "0",
 "space": "no",
 "version": "2.4.0"
}
]


In [32]:
#comet-score
!comet-score -s medline_en-de.en.txt -t translation_finetuned_mBART.txt -r medline_en-de_reference_de.txt

Seed set to 1
Fetching 5 files: 100% 5/5 [00:00<00:00, 55188.21it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 4/4 [00:03<00:00,  1.16it/s]
translation_finetuned_mBART.txt	Segment 0	score: 0.7825
translation_finetuned_mBART.txt	Segment 1	score: 0.9100
translation_finetune

##Results fine-tuned model

bleu: 24.8

chrF: 47.9

comet: 0.721

In [6]:
#compute BLEU and chrF reference with original facebook model
!sacrebleu medline_en-de.de.txt -l en-de -i translation_baseline_mBART.txt  -m bleu chrf

[
{
 "name": "BLEU",
 "score": 23.8,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.0",
 "verbose_score": "48.3/26.8/18.6/13.3 (BP = 1.000 ratio = 1.020 hyp_len = 1252 ref_len = 1228)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.0"
},
{
 "name": "chrF2",
 "score": 46.8,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:0|space:no|version:2.4.0",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "0",
 "space": "no",
 "version": "2.4.0"
}
]


In [33]:
!comet-score -s medline_en-de.en.txt -t translation_baseline_mBART.txt -r medline_en-de_reference_de.txt

Seed set to 1
Fetching 5 files: 100% 5/5 [00:00<00:00, 46500.04it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 4/4 [00:04<00:00,  1.00s/it]
translation_baseline_mBART.txt	Segment 0	score: 0.6589
translation_baseline_mBART.txt	Segment 1	score: 0.9065
translation_baseline_m

##Results baseline model

bleu: 23.8

chrF: 46.8

comet: 0.705

In [7]:
#compute statistical significance with paired bootstraping
!sacrebleu medline_en-de.de.txt -l en-de -i translation_baseline_mBART.txt translation_finetuned_mBART.txt -m bleu --paired-bs

sacreBLEU: Found 2 systems.
sacreBLEU: Pre-computing BLEU statistics for 'Baseline: translation_baseline_mBART.txt'
sacreBLEU: Computing BLEU for 'translation_finetuned_mBART.txt' and extracting sufficient statistics
sacreBLEU:  > Performing paired bootstrap resampling test (# resamples: 1000)
[
    {
        "system": "Baseline: translation_baseline_mBART.txt",
        "BLEU": {
            "score": 23.789414401473216,
            "p_value": null,
            "mean": 23.47321659188576,
            "ci": 7.189145969834421
        }
    },
    {
        "system": "translation_finetuned_mBART.txt",
        "BLEU": {
            "score": 24.841132821115885,
            "p_value": 0.13186813186813187,
            "mean": 24.619515441216777,
            "ci": 7.6386377131164345
        }
    }
]


p_value --> when testing for Null Hypothesis (= system and baseline are the same: finetuning didnt help) - you reject it or accept it - p_value is roughly roughly probability of the absolute difference (how probable it is that your attempt was significant --> improvement was SLIGHTLY betterbut not good enough to be seen as an actual improvement --> finetuning was not statistically significant)

if p < 0.05 --> NULL Hypothesis is rejected. --> finetuning did help